## Data format to PIF converter
Let's write an converter to map the Instron output file to a PIF file.

In [1]:
with open("example_data/RT-loadtofail_10e-3.txt") as f:
    lines = f.readlines()
    print '\n'.join(lines[:15])

MTS793|MPT|ENU|1|2|.|/|:|1|0|0|A



Data Acquisition						Time:	102.54224	Sec	7/5/2010 1:24:13 PM

Axial Displacement	Axial Force	Axial Strain	Time

mm	N	mm/mm	Sec

0.001254817	-1.2510437	-0.18667668	0.13891602

0.00041648507	-2.2480204	-0.18673906	0.23901367

-0.00032296643	-3.24281	-0.18673992	0.33911133

-4.4806453e-05	-3.5292149	-0.18668246	0.43920898

-0.00031411514	-2.728333	-0.18660255	0.53930664

-0.00014413759	-1.8402346	-0.18657748	0.6394043

0.0011240146	-1.2426991	-0.18662354	0.73950195

0.0007610707	-1.3819313	-0.18669833	0.83959961

0.0004964745	-2.4153166	-0.18674521	0.93969727

0.00013828411	-3.4589834	-0.18672836	1.0397949



Using a given analysis script to compile the force, displacement, stress, and strain, and using those values to calculate the elastic modulus and critical stress (thanks Saurabh!), we can package the Instron output into the PIF format.

In [2]:
from thermo_mechanical_analysis import Thermo
from pypif import pif
from pypif.obj import *

def instron_to_pif(instron_file, material_name):
    instron = Thermo(instron_file)
    instron.extract_data()
    instron.calc_elastic_modulus()
    system = ChemicalSystem(name=material_name)

    elastic_modulus = Property(name="Elastic Modulus", units="MPa", scalars=instron.elastic_modulus)
    critical_stress = Property(name="Critical Stress", units="MPa", scalars=instron.critical_stress)
    stress_strain = Property(name="Stress", units="MPa", scalars=instron.stress,
                             conditions=Value(name="Strain", units="mm/mm", scalars=instron.strain))
    force_displacement = Property(name="Force", units="N", scalars=instron.force,
                                  conditions=Value(name="Displacement", units="mm", scalars=instron.displacement))
    system.properties = [elastic_modulus, critical_stress, stress_strain, force_displacement]
    return system

Now, upload the result http://citrination.com/pif/150713/1/instron_example

In [3]:
from citrination_client import CitrinationClient
from os import environ

client = CitrinationClient(environ['CITRINATION_API_KEY'], 'https://citrination.com')
dataset_id = 150713
instron = instron_to_pif("example_data/RT-loadtofail_10e-3.txt", "Sample")
instron.uid = "instron_example"

with open("example_data/instron.json", 'w') as fp:
    pif.dump(instron, fp)
    
client.upload_file("example_data/instron.json", dataset_id)

Elastic Modulus:78286.464381
Critical Stress:677.7138222


'{"data_set_id": "150713", "message": "Upload is complete.", "version": 1}'